# Yandex.Music

## Introducción <a id='intro'></a>
Siempre que investiguemos, necesitamos formular hipótesis que después podamos probar. A veces aceptamos estas hipótesis; otras, las rechazamos. Para tomar las decisiones correctas, una empresa debe ser capaz de entender si está haciendo las suposiciones correctas.

En este proyecto, compararás las preferencias musicales de las ciudades de Springfield y Shelbyville. Estudiarás datos reales de Yandex.Music para probar las hipótesis de abajo y comparar el comportamiento del usuario de esas dos ciudades.

### Objetivo:
Prueba tres hipótesis:
1. La actividad de los usuarios difiere según el día de la semana y dependiendo de la ciudad.
2. Los lunes por la mañana, los habitantes de Springfield y Shelbyville escuchan diferentes géneros. Lo mismo ocurre con los viernes por la noche.
3. Los oyentes de Springfield y Shelbyville tienen preferencias distintas. En Springfield prefieren el pop mientras que en Shelbyville hay más aficionados al rap.

### Etapas
Los datos del comportamiento del usuario se almacenan en el archivo `/datasets/music_project_en.csv`. No hay ninguna información sobre la calidad de los datos así que necesitarás examinarlos antes de probar las hipótesis.

Primero, evaluarás la calidad de los datos y verás si los problemas son significativos. Entonces, durante el preprocesamiento de datos, tomarás en cuenta los problemas más críticos.

## Etapa 1. Descripción de los datos <a id='data_review'></a>


In [ ]:
# importando pandas
import pandas as pd

In [ ]:
# leyendo el archivo y almacenándolo en df
df = pd.read_csv('/datasets/music_project_en.csv')
df.describe()

,userID,Track,artist,genre,City,time,Day
count,65079,63736,57512,63881,65079,65079,65079
unique,41748,39666,37806,268,2,20392,3
top,A8AE9169,Brand,Kartvelli,pop,Springfield,08:14:07,Friday
freq,76,136,136,8850,45360,14,23149


In [ ]:
# obteniendo las 10 primeras filas de la tabla df
print(df.head(10))

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                       Chains          Obladaet  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE                     L’estate       Julia Dalia  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

        City        time        Day  
0  Shelbyville  20:28:33  Wednesday  
1  Springfield  14:07:09     Friday  
2  Shelbyville  20:58:07  Wednesday  
3  Shelbyville  08:37:09     Monday  
4  Springfield  08:34:34     Monday  
5

In [ ]:
# obteniendo información general sobre los datos en df
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB
None


La tabla contiene siete columnas. Todas almacenan el mismo tipo de datos: objeto.

De acuerdo con la documentación:
- `'userID'` — identificador del usuario
- `'Track'` — título de la pista
- `'artist'` — nombre del artista
- `'genre'` — género
- `'City'` — ciudad del usuario
- `'time'` — el periodo de tiempo exacto en que se reprodujo la pista
- `'Day'` — día de la semana

Podemos ver tres problemas con el estilo en los nombres de las columnas:
1. Algunos nombres están en mayúsculas, otros en minúsculas.
2. Hay algunos espacios en algunos nombres.
3. Time no dice en que se mide el tiempo.
4. El número de valores de las columnas es diferente. Esto significa que los datos contienen valores ausentes.


### Conclusiones <a id='data_review_conclusions'></a>

Cada fila de la tabla almacena datos de la pista que fue reproducida. Algunas columnas describen la pista en sí: su título, el artista y el género. El resto transmite la información del usuario: la ciudad de la que viene, el tiempo que ha reproducido la pista.

Está claro que los datos son suficientes para probar la hipótesis. Sin embargo, hay valores ausentes.

Para continuar, necesitamos preprocesar los datos.

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing'></a>


### Estilo del encabezado <a id='header_style'></a>

In [ ]:
# la lista de los nombres de las columnas en la tabla df
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


In [ ]:
# renombra las columnas
df=df.rename(
    columns={
        '  userID': 'user_id',
        'Track': 'track',
        'artist': 'artist',
        'genre': 'genre',
        '  City  ': 'city',
        'time': 'time',
        'Day': 'day',
    }
)


In [ ]:
# comprobando el resultado: la lista de los nombres de las columnas
print (df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


### Valores ausentes <a id='missing_values'></a>


In [ ]:
# calculando valores ausentes
print(df.isna().sum(),'\n')
print(df.isnull().sum())


user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64 

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64


Se reemplaza los valores ausentes en `'track'`, `'artist'`, y `'genre'` con la string `'unknown'`. Para ello, se crea la lista `columns_to_replace`, se recorre con un bucle `for` y se reemplaza los valores ausentes en cada una de las columnas:

In [ ]:
# recorriendo los nombres de las columnas y reemplazando los valores ausentes con 'unknown'

def columns_to_replace(absent_values, correct_value):
   for absent_value in absent_values:
        df['artist']= df['artist'].replace(absent_values, correct_value)
        df['track']= df['track'].replace(absent_values, correct_value)
        df['genre']= df['genre'].replace(absent_values, correct_value)

value= [None]
word= 'unknown'
columns_to_replace(value, word)

In [ ]:
# contando valores ausentes
print(df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Duplicados <a id='duplicates'></a>


In [ ]:
# contando duplicado obvios

duplicated_df = df[df.duplicated()]
print(duplicated_df)

        user_id                track                          artist  \
575    E7F07B46                Crazy                  The Manhattans   
832    7671A47A           Johnny Go!                          NikitA   
1216   69467B01        Change It All                  Harrison Storm   
1754   13B1A573    Te Adoramos Jesús                         Athenas   
1964   B24668A0  Mad over You Mashup                      Nana Fofie   
...         ...                  ...                             ...   
65042  83E9C8C4        Buddhist Beat  Asian Zen Spa Music Meditation   
65056  2E25BCD2          Psychobitch                          Easter   
65059  F231C47E  All Summer in a Day                     VHS Or BETA   
65067  F1B93F29          Poison Kiss                     Centerstone   
65069  BE1AAD74            Waterwalk                Eduardo Gonzales   

            genre         city      time        day  
575           rnb  Springfield  13:39:46     Monday  
832        ruspop  Springfi

In [ ]:
# eliminando duplicados obvios

df = df.drop_duplicates()

In [ ]:
# comprobando duplicados

print(df.duplicated().sum())

0


In [ ]:
# inspeccionando los nombres de géneros únicos

list_genre = df['genre']
sorted_list_genre = list_genre.sort_values()

print(sorted_list_genre.unique())


['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'ambient' 'americana' 'animated' 'anime' 'arabesk' 'arabic' 'arena'
 'argentinetango' 'art' 'audiobook' 'avantgarde' 'axé' 'baile' 'balkan'
 'beats' 'bigroom' 'black' 'bluegrass' 'blues' 'bollywood' 'bossa'
 'brazilian' 'breakbeat' 'breaks' 'broadway' 'cantautori' 'cantopop'
 'canzone' 'caribbean' 'caucasian' 'celtic' 'chamber' 'children' 'chill'
 'chinese' 'choral' 'christian' 'christmas' 'classical' 'classicmetal'
 'club' 'colombian' 'comedy' 'conjazz' 'contemporary' 'country' 'cuban'
 'dance' 'dancehall' 'dancepop' 'dark' 'death' 'deep' 'deutschrock'
 'deutschspr' 'dirty' 'disco' 'dnb' 'documentary' 'downbeat' 'downtempo'
 'drum' 'dub' 'dubstep' 'eastern' 'easy' 'electronic' 'electropop' 'emo'
 'entehno' 'epicmetal' 'estrada' 'ethnic' 'eurofolk' 'european'
 'experimental' 'extrememetal' 'fado' 'film' 'fitness' 'flamenco' 'folk'
 'folklore' 'folkmetal' 'folkrock' 'folktronica' 'forró' 'frankreich'
 'französisch' 

In [ ]:
# función para reemplazar duplicados implícitos

def replace_wrong_genres(wrong_genres, correct_genre):
   for wrong_genre in wrong_genres:
        df['genre']= df['genre'].replace(wrong_genres, correct_genre)

In [ ]:
# eliminando duplicados implícitos

word= ['hip', 'hop', 'hip-hop']
correct= 'hiphop'
replace_wrong_genres(word, correct)

In [ ]:
# revisando en busca de duplicados implícitos

print(df['genre'].sort_values().unique())

['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'ambient' 'americana' 'animated' 'anime' 'arabesk' 'arabic' 'arena'
 'argentinetango' 'art' 'audiobook' 'avantgarde' 'axé' 'baile' 'balkan'
 'beats' 'bigroom' 'black' 'bluegrass' 'blues' 'bollywood' 'bossa'
 'brazilian' 'breakbeat' 'breaks' 'broadway' 'cantautori' 'cantopop'
 'canzone' 'caribbean' 'caucasian' 'celtic' 'chamber' 'children' 'chill'
 'chinese' 'choral' 'christian' 'christmas' 'classical' 'classicmetal'
 'club' 'colombian' 'comedy' 'conjazz' 'contemporary' 'country' 'cuban'
 'dance' 'dancehall' 'dancepop' 'dark' 'death' 'deep' 'deutschrock'
 'deutschspr' 'dirty' 'disco' 'dnb' 'documentary' 'downbeat' 'downtempo'
 'drum' 'dub' 'dubstep' 'eastern' 'easy' 'electronic' 'electropop' 'emo'
 'entehno' 'epicmetal' 'estrada' 'ethnic' 'eurofolk' 'european'
 'experimental' 'extrememetal' 'fado' 'film' 'fitness' 'flamenco' 'folk'
 'folklore' 'folkmetal' 'folkrock' 'folktronica' 'forró' 'frankreich'
 'französisch' 

### Conclusiones <a id='data_preprocessing_conclusions'></a>
Detectamos tres problemas con los datos:

- Estilos de encabezados incorrectos
- Valores ausentes
- Duplicados obvios e implícitos

Los encabezados han sido eliminados para conseguir que el procesamiento de la tabla sea más sencillo.

Todos los valores ausentes han sido reemplazados por `'unknown'`. Pero todavía tenemos que ver si los valores ausentes en `'genre'` afectan a nuestros cálculos.

La ausencia de duplicados hará que los resultados sean mas precisos y fáciles de entender.

Ahora ya podemos continuar probando las hipótesis.

## Etapa 3. Prueba de hipótesis <a id='hypotheses'></a>

### Hipótesis 1: comparar el comportamiento del usuario en las dos ciudades <a id='activity'></a>

De acuerdo con la primera hipótesis, los usuarios de Springfield y Shelbyville escuchan música de forma distinta. Comprueba esto utilizando los datos de tres días de la semana: lunes, miércoles y viernes.

* Se divide a los usuarios en grupos por ciudad.
* Se compara cuántas pistas reprodujo cada grupo el lunes, el miércoles y el viernes.


In [ ]:
# contando las pistas reproducidas en cada ciudad

count_track = df.groupby('city')['track'].count()
print(count_track)

city
Shelbyville    18512
Springfield    42741
Name: track, dtype: int64


Springfield ha reproducido más pistas que Shelbyville. Pero eso no implica que los ciudadanos de Springfield escuchen música más a menudo. Esta ciudad es simplemente más grande y hay más usuarios.

In [ ]:
# calculando las pistas reproducidas en cada uno de los tres días

track_day = df.groupby('day')['track'].count()
print(track_day)

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64


El miércoles fue el día más silencioso de todos. Pero si consideramos las dos ciudades por separado podríamos llegar a una conclusión diferente.

In [ ]:
# <creando la función number_tracks()>
# declararemos la función con dos parámetros: day=, city=.

def number_tracks(day, city):
    #track_list = df[(df['day'] == day) & (df['city']==city)]
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city']==city]

    track_list_count = track_list['user_id'].count()

    return track_list_count


In [ ]:
# el número de canciones reproducidas en Springfield el lunes

tracks_Spr_Mon = number_tracks('Monday', 'Springfield')
print (tracks_Spr_Mon)

15740


In [ ]:
# el número de canciones reproducidas en Shelbyville el lunes

tracks_Shel_Mon = number_tracks('Monday', 'Shelbyville')
print (tracks_Shel_Mon)

5614


In [ ]:
# el número de canciones reproducidas en Springfield 110iércoles

tracks_Spr_Wed = number_tracks('Wednesday', 'Springfield')
print (tracks_Spr_Wed)

11056


In [ ]:
# el número de canciones reproducidas en Shelbyville el miércoles

tracks_Shel_Wed = number_tracks('Wednesday', 'Shelbyville')
print (tracks_Shel_Wed)

7003


In [ ]:
# el número de canciones reproducidas en Springfield el viernes

tracks_Spr_Fri = number_tracks('Friday', 'Springfield')
print (tracks_Spr_Fri)

15945


In [ ]:
# el número de canciones reproducidas en Shelbyville el viernes

tracks_Shel_Fri = number_tracks('Friday', 'Shelbyville')
print (tracks_Shel_Fri)

5895


In [ ]:
#import pandas as pd

tracks_city = [
    ['Springfield', 15740, 11056, 15945],
    ['Shelbyville', 5614, 7003, 5895]
]

data_col = ['city', 'monday', 'wednesday', 'friday']

tracks_table = pd.DataFrame(data = tracks_city, columns = data_col)

In [ ]:
# tabla con los resultados

print(tracks_table)

          city  monday  wednesday  friday
0  Springfield   15740      11056   15945
1  Shelbyville    5614       7003    5895


**Conclusiones**

Los datos revelan las diferencias en el comportamiento de los usuarios:

- En Springfield, el número de canciones reproducidas alcanzan el punto máximo los lunes y viernes mientras que los miércoles hay un descenso de la actividad.
- En Shelbyville, al contario, los usuarios escuchan más música los miércoles. La actividad de los usuarios los lunes y viernes es menor.

Así que la primera hipótesis parece ser correcta.

### Hipótesis 2: música al principio y al final de la semana <a id='week'></a>

De acuerdo con la segunda hipótesis, los lunes por la mañana y los viernes por la noche los ciudadanos de Springfield escuchan géneros que difieren de aquellos que los usuarios de Shelbyville disfrutan.

Obtén tablas (asegúrate de que el nombre de tu tabla combinada encaja con el DataFrame dado en los dos bloques de código de abajo):
* Para Springfield — `spr_general`
* Para Shelbyville — `shel_general`

In [ ]:
# obteniendo la tabla spr_general de las filas de df,
# donde los valores en la columna 'city' es 'Springfield'

spr_general = df[df['city'] == "Springfield"]
print(spr_general)


In [ ]:
        user_id                          track                   artist  \
1      55204538    Delayed Because of Accident         Andreas Rönnberg
4      E2DC1FAE                    Soul People               Space Echo
6      4CB90AA5                           True             Roman Messer
7      F03E1C1F               Feeling This Way          Polina Griffith
8      8FA1D3BE                       L’estate              Julia Dalia
...         ...                            ...                      ...
65073  83A474E7  I Worship Only What You Bleed  The Black Dahlia Murder
65074  729CBB09                        My Name                   McLean
65076  C5E3A0D5                      Jalopiina                  unknown
65077  321D0506                  Freight Train            Chas McDevitt
65078  3A64EF84      Tell Me Sweet Little Lies             Monica Lopez

              genre         city      time        day
1              rock  Springfield  14:07:09     Friday
4             dance  Springfield  08:34:34     Monday
6             dance  Springfield  13:00:07  Wednesday
7             dance  Springfield  20:47:49  Wednesday
8            ruspop  Springfield  09:17:40     Friday
...             ...          ...       ...        ...
65073  extrememetal  Springfield  21:07:12     Monday
65074           rnb  Springfield  13:32:28  Wednesday
65076    industrial  Springfield  20:09:26     Friday
65077          rock  Springfield  21:43:59     Friday
65078       country  Springfield  21:59:46     Friday

[42741 rows x 7 columns]

In [ ]:
# obteniendo shel_general de las filas df,
# donde el valor de la columna 'city' es 'Shelbyville'

shel_general = df[df['city'] == "Shelbyville"]
print(shel_general)

In [ ]:
  user_id                              track              artist  \
0      FFB692EC                  Kamigata To Boots    The Mass Missile
2        20EC38                  Funiculì funiculà         Mario Lanza
3      A3DD03C9              Dragons in the Sunset          Fire + Ice
5      842029A1                             Chains            Obladaet
9      E772D5C0                          Pessimist             unknown
...         ...                                ...                 ...
65063  D94F810B        Theme from the Walking Dead  Proyecto Halloween
65064  BC8EC5CF       Red Lips: Gta (Rover Rework)               Rover
65065  29E04611                       Bre Petrunko       Perunika Trio
65066  1B91C621             (Hello) Cloud Mountain     sleepmakeswaves
65075  D08D4A55  Maybe One Day (feat. Black Spade)         Blu & Exile

            genre         city      time        day
0            rock  Shelbyville  20:28:33  Wednesday
2             pop  Shelbyville  20:58:07  Wednesday
3            folk  Shelbyville  08:37:09     Monday
5          rusrap  Shelbyville  13:09:41     Friday
9           dance  Shelbyville  21:20:49  Wednesday
...           ...          ...       ...        ...
65063        film  Shelbyville  21:14:40     Monday
65064  electronic  Shelbyville  21:06:50     Monday
65065       world  Shelbyville  13:56:00     Monday
65066    postrock  Shelbyville  09:22:13     Monday
65075      hiphop  Shelbyville  10:00:00     Monday

[18512 rows x 7 columns]

In [ ]:
# declarando la función genre_weekday() con los parámetros day=, time1= y time2=. Debería
# devolver información sobre los géneros más populares de un determinado día a una determinada hora:

def genre_weekday(df, day, time1, time2):

    # filtrado consecutivo
    # genre_df solo almacenará aquellas filas df en las que el día sea igual a day=
    genre_df = df[df['day'] == day]    # escribe tu código aquí

    # genre_df solo almacenará aquellas filas df en las que el tiempo sea menos que time2=
    genre_df = genre_df[genre_df['time'] < time2]    # escribe tu código aquí

    # genre_df solo almacenará aquellas filas df en las que el tiempo sea mayor que time1=
    genre_df = genre_df[genre_df['time'] > time1]    # escribe tu código aquí

    # agrupa el DataFrame filtrado por la columna con los nombres de los géneros, toma la columna de género, y encuentra el número de filas por cada género con el método count()
    #genre_df_grouped = genre_df['genre'].count()  # escribe tu código aquí
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()

    # ordenaremos el resultado en orden descendente (por lo que los géneros más populares aparecerán primero en el objeto Series)
    genre_df_sorted = genre_df_grouped.sort_values(ascending = False)  # escribe tu código aquí

    # devolveremos el objeto Series que almacena los 15 géneros más populares en un día determinado en un periodo de tiempo determinado
    return genre_df_sorted[:15]


In [ ]:
# llamando a la función para el lunes por la mañana en Springfield (utilizando spr_general en vez de la tabla df)

spr_mon_morn = genre_weekday(spr_general,'Monday', '07:00', '11:00')
print(spr_mon_morn)


In [ ]:
genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
classical      157
metal          120
jazz           100
folk            97
soundtrack      95
Name: genre, dtype: int64

In [ ]:
# llamando a la función para el lunes por la mañana en Shelbyville (utilizando shel_general en vez de la tabla df)

shel_mon_morn = genre_weekday(shel_general,'Monday', '07:00', '11:00')
print(shel_mon_morn)


In [ ]:
genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
world           36
rap             32
soundtrack      31
rnb             27
metal           27
Name: genre, dtype: int64

In [ ]:
# llamando a la función para el viernes por la tarde en Springfield

spr_fri_aft = genre_weekday(spr_general,'Friday', '17:00', '23:00')
print(spr_fri_aft)


In [ ]:
genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
jazz           111
unknown        110
soundtrack     105
rnb             90
metal           88
Name: genre, dtype: int64

In [ ]:
# llamando a la función para el viernes por la tarde en Shelbyville

shel_fri_aft = genre_weekday(shel_general,'Friday', '17:00', '23:00')
print(shel_fri_aft)


In [ ]:
genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
unknown         47
ruspop          47
soundtrack      40
metal           39
rap             36
Name: genre, dtype: int64

**Conclusión**

Habiendo comparado los 15 géneros más populares del lunes por la mañana podemos concluir lo siguiente:

1. Los usuarios de Springfield y Shelbyville escuchan música similar. Los cinco géneros más populares son los mismos, solo rock y electrónica han intercambiado posiciones.

2. En Springfield el número de valores ausentes resultaron ser tan altos que el valor `'unknown'` llegó al décimo. Esto significa que los valores ausentes forman una parte considerable de los datos, lo que podría ser la base de la cuestión sobre la fiabilidad de nuestras conclusiones.

Para el viernes por la tarde, la situación es similar. Los géneros individuales varían algo pero, en general, los 15 más populares son parecidos en las dos ciudades.

De esta forma, la segunda hipótesis ha sido parcialmente demostrada:
* Los usuarios escuchan música similar al principio y al final de la semana.
* No hay una gran diferencia entre Springfield y Shelbyville. En ambas ciudades, el pop es el género más popular.

Sin embargo, el número de valores ausentes hace este resultado un tanto cuestionable. En Springfield, hay tantos que afectan a nuestros 15 más populares. De no faltarnos esos valores, las cosas podrían parecer diferentes.

### Hipótesis 3: preferencias de género en Springfield y Shelbyville <a id='genre'></a>

Hipótesis: Shelbyville ama la música rap. A los ciudadanos de Springfield les gusta más el pop.

In [ ]:
# en una línea: agrupa la tabla spr_general por la columna 'genre',

spr_general_genre = spr_general.groupby('genre')['genre']

# cuenta los valores 'genre' con count() en la agrupación,

spr_general_count = spr_general_genre.count()

# ordena el Series resultante en orden descendiente, y almacénalo en spr_genres

spr_genres = spr_general_count.sort_values(ascending=False)

In [ ]:
# imprimiendo las 10 primeras filas de spr_genres

print (spr_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


In [ ]:
# en una línea: agrupa la tabla shel_general por la columna 'genre',

shel_general_genre = shel_general.groupby('genre')['genre']

# cuenta los valores 'genre' en el agrupamiento con count(),

shel_general_count = shel_general_genre.count()

# ordena el Series resultante en orden descendente y guárdalo en shel_genres

shel_genres = shel_general_count.sort_values(ascending=False)

In [ ]:
# imprimiendo las 10 primeras filas de shel_genres

print (shel_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Conclusión**

La hipótesis ha sido parcialmente demostrada:
* La música pop es el género más popular en Springfield, tal como se esperaba.
* Sin embargo, la música pop ha resultado ser igual de popular en Springfield que en Shelbyville y el rap no estaba entre los 5 más populares en ninguna de las ciudades.


# Conclusiones <a id='end'></a>

Hemos probado las siguientes tres hipótesis:

1. La actividad de los usuarios difiere dependiendo del día de la semana y de las distintas ciudades.
2. Los lunes por la mañana los residentes de Springfield y Shelbyville escuchan géneros distintos. Lo mismo ocurre con los viernes por la noche.
3. Los oyentes de Springfield y Shelbyville tienen distintas preferencias. En ambas ciudades, Springfield y Shelbyville, se prefiere el pop.

Tras analizar los datos, concluimos:

1. La actividad del usuario en Springfield y Shelbyville depende del día de la semana aunque las ciudades varían de diferentes formas.

La primera hipótesis ha sido aceptada completamente.

2. Las preferencias musicales no varían significativamente en el transcurso de la semana en Springfield y Shelbyville. Podemos observar pequeñas diferencias en el orden los lunes, pero:
* En Springfield y Shelbyville la gente lo que más escucha es la música pop.

Así que no podemos aceptar esta hipótesis. También debemos tener en cuenta que el resultado podría haber sido diferente si no fuera por los valores ausentes.

3. Resulta que las preferencias musicales de los usuarios de Springfield y Shelbyville son bastante parecidas.

La tercera hipótesis es rechazada. Si hay alguna diferencia en las preferencias no se puede observar en los datos.

### Nota
En los proyectos reales, la investigación supone el estudio de hipótesis estadísticas que es más preciso y cuantitativo. También ten en cuenta que no siempre podemos sacar conclusiones sobre una ciudad entera basándonos en datos de una sola fuente.

Analizarás el estudio de hipótesis en el sprint de análisis estadístico de datos.